In [ ]:
#dataset init

#dataset description: https://github.com/harrylclc/RefD-dataset/
global_warming = pd.read_csv('/content/drive/My Drive//RefD-dataset-master/CrowdComp/CrowdComp_Global_warming_Prerequisite_HIT.csv', warn_bad_lines=True, error_bad_lines=False)
meiosis = pd.read_csv('/content/drive/My Drive//RefD-dataset-master/CrowdComp/CrowdComp_Meiosis_Prerequisite_HIT.csv', warn_bad_lines=True, error_bad_lines=False)
newton = pd.read_csv('/content/drive/My Drive//RefD-dataset-master/CrowdComp/CrowdComp_Newton_Laws_Prerequisite_HIT.csv', warn_bad_lines=True, error_bad_lines=False)
parallel_postulate = pd.read_csv('/content/drive/My Drive//RefD-dataset-master/CrowdComp/CrowdComp_Parallel_Postulate_Prerequisite_HIT.csv', warn_bad_lines=True, error_bad_lines=False)
pk_cryptography = pd.read_csv('/content/drive/My Drive//RefD-dataset-master/CrowdComp/CrowdComp_PublicKeyCryptography_Prerequisite_HIT.csv', warn_bad_lines=True, error_bad_lines=False)
#merge the datasets
df = pd.concat([global_warming,meiosis,newton,parallel_postulate,pk_cryptography])
#each A B pair in this dataset has multiple possible answers because multiple people gave an answer on the same pair
#possible answers are:
#A is prerequisite of B
#B is prerequisite of A
#other (unrelated, related but not prereq, don't know, etc)
#we will take as true the answer with most votes
#on equal amount of answers we consider A and B unrelated
#to do this we use a dictionary where the key is A+B 
#and the value is the number of votes for each answer and the names of A and B
#dict[A+B] = [A,B,AB,BA,unrelated]
#AB = A is prereq of B 
#BA = B is prereq of A
dict_votes = {}
for index,row in df.iterrows():
  A = row['Input.SourceConceptBase']
  B = row['Input.TargetConceptBase']
  key =  A+B
  AB = 0
  BA = 0
  unrelated = 0
  if str(row['Answer.Source2Target']) == 'Source2Target':
    AB = 1
  elif str(row['Answer.Target2Source']) == 'Target2Source':
    BA = 1
  else:
    unrelated = 1  

  if key not in dict_votes:
    dict_votes[key] = [A,B,AB,BA,unrelated]
  else:
    dict_votes[key][2] = dict_votes[key][2] + AB
    dict_votes[key][3] = dict_votes[key][3] + BA
    dict_votes[key][4] = dict_votes[key][4] + unrelated

#now we iterate the values of the dictionary and create a dataset A B 0/1
dataset = {'A':[],'B':[],'prerequisite':[]}
for key, value in dict_votes.items():
  A = value[0]
  B = value[1]
  votes = [value[2],value[3],value[4]]
  m = max(votes)
  position_of_max_value = [i for i, j in enumerate(votes) if j == m]
  #if there's not unique max vote or if the max vote is unrelated
  if len(position_of_max_value) > 1 or position_of_max_value[0] == 2:
    dataset['A'].append(A)
    dataset['B'].append(B)
    dataset['prerequisite'].append(0)
    dataset['A'].append(B)
    dataset['B'].append(A)
    dataset['prerequisite'].append(0)
  elif position_of_max_value[0] == 0:
    dataset['A'].append(A)
    dataset['B'].append(B)
    dataset['prerequisite'].append(1)
    dataset['A'].append(B)
    dataset['B'].append(A)
    dataset['prerequisite'].append(0)
  elif position_of_max_value[0] == 1:
    dataset['A'].append(A)
    dataset['B'].append(B)
    dataset['prerequisite'].append(0)
    dataset['A'].append(B)
    dataset['B'].append(A)
    dataset['prerequisite'].append(1)

df = pd.DataFrame.from_dict(dataset) 
#we shuffle the dataframe since right now it would give trouble while creating a proper training and validation set
df = df.sample(frac=1)

#train and validatin split
train = df.sample(frac=0.8,random_state=200) 
validation = df.drop(train.index)

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('en',extract_format=wikipediaapi.ExtractFormat.WIKI)

#create dictionary of tokenized documents
punct = string.punctuation + '«``»' + "''"
en_stop_words = nltk.corpus.stopwords.words('english')

doc_dict = {}

#we get the english wikipedia pages

for index,row in df.iterrows():
  #Titles of the two documents
  title_A = row[0]
  title_B = row[1]
  
  #we check if the page related to the titles is already in the dictionary
  if title_A not in doc_dict:
    p_wiki = wiki_wiki.page(title_A)
    document = (p_wiki.title + " " + p_wiki.text).replace("'"," ")
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = (p_wiki.title  + p_wiki.text).replace("'"," ")
    document = document.replace("\\"," ")
    document = document.replace("displaystyle"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[title_A] = no_stop_words

  if title_B not in doc_dict:
    p_wiki = wiki_wiki.page(title_B)
    document = (p_wiki.title + " " + p_wiki.text).replace("'"," ")
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = (p_wiki.title  + p_wiki.text).replace("'"," ")
    document = document.replace("\\"," ")
    document = document.replace("displaystyle"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[title_B] = no_stop_words

print(doc_dict)   